## 1 Import libraries and create user-defined functions

In [1]:
# authentication
import snowflake.connector
import configparser
config = configparser.ConfigParser()
config.read('config.ini')

# data manipulation
import pandas as pd
import numpy as np

# visualization
import matplotlib.pyplot as plt
import seaborn as sns

# prevent pandas column assignment warning
pd.options.mode.chained_assignment = None  # default='warn'

from sql_queries import rates_query

In [2]:
def connect_to_db():

    try:
        conn = snowflake.connector.connect(
        user=config['snowflake']['user'],
        account=config['snowflake']['account'],
        authenticator='externalbrowser',
        database=config['snowflake']['database']
        )
        print('Connected successfully')
    except:
        print('Connection failed, try again')

    return conn

def run_sql_query(conn,sql_query,params,return_format):
    
    results = None
    cursor = conn.cursor()
    if params != None:
        query = cursor.execute(sql_query,params)
    else:
        query = cursor.execute(sql_query)

    # list of tuples
    if return_format == 0:
        results = []
        rows = cursor.fetchall()
        for row in rows:
            results.append(row)

    # pandas dataframe
    if return_format == 1:
        try:
            cols = [column[0] for column in query.description]
            results = pd.DataFrame.from_records(data=query.fetchall(),columns=cols)
        except:
            pass

    # numpy array
    if return_format == 2:
        rows = cursor.fetchall()
        results = np.array(rows)
        
    return results

# 2 Connect to Snowflake

In [ ]:
conn = connect_to_db()

# 3 Run SQL queries to find field values

## 3.1 Filing Types

In [4]:
sql_get_filing_types = '''select distinct FILINGTYPE_16 from XPRESSFEED.SNL_PC_INSURANCEPRODUCTFILINGS_GENERALINFORMATION;'''
filing_types = run_sql_query(conn,sql_query=sql_get_filing_types,params=None,return_format=0)
for ft in filing_types:
    print(ft[0])

Form,Rate,Rule,
Form,Rule,
Form,Rate,RF-3,Rule,
Other,
Rate,Transferred Pro
Rate,
Form,Rate,
New Program,
Advertising,
Transferred Program,
RF-3,
Rate,Rule,
Form,
Rule,
Advertising,Form,Rat


## 3.2 Types of insurance

In [14]:
sql_get_tois = '''select distinct TYPEOFINSURANCEDESCRIPTION_8 from XPRESSFEED.SNL_PC_INSURANCEPRODUCTFILINGS_GENERALINFORMATION ORDER BY TYPEOFINSURANCEDESCRIPTION_8 asc;'''

In [15]:
tois = run_sql_query(conn,sql_query=sql_get_tois,params=None,return_format=0)
for toi in tois:
    print(toi[0])

Aircraft
Auto Warranty
Bailbonds
Boiler and Machinery or Equipment Breakdown
Burglary and Theft
CMP Liability Portion Only
CMP Liability and Non-Liability
CMP Non-Liability Portion Only
Commercial Auto
Credit
Credit - Personal Property
Credit-Credit Default
Crop
Dwelling Fire/Personal Liability
Earthquake
Farmowners
Fidelity
Fidelity and Surety
Financial Guaranty
Flood
Glass
Home Warranty
Homeowner/ Auto Combinations
Homeowners
Industrial Fire
Inland Marine
Interline Filings
Livestock & Live Animals
Med Mal-Claims Made Only
Med Mal-Claims Made and Occurrence
Med Mal-Occurrence Only
Misc Casualty - Commercial
Misc Casualty - Personal
Mobile Home Physical Damage
Mobile Homes under Transport
Mortgage Guaranty
Motor Vehicle Service Contracts
Ocean Marine
Other
Other Liability-Claims Made Only
Other Liability-Occ Only
Other Liability-Occ/Claims Made
Other Lines of Business
Other Warranty
Personal Auto
Personal Farmowners
Premium Finance
Prepaid Legal
Product Liability
Property
Service Warra

## 3.3 Sub-types of insurance 

In [16]:
toi = 'Personal Auto'
sql_get_subtois = '''select distinct SUBTYPEOFINSURANCEDESCRIPTION_10 from XPRESSFEED.SNL_PC_INSURANCEPRODUCTFILINGS_GENERALINFORMATION where TYPEOFINSURANCEDESCRIPTION_8 = %s;'''

In [17]:
sub_tois = run_sql_query(conn,sql_query=sql_get_subtois,params=(toi,),return_format=0)
for sub_toi in sub_tois:
    print(sub_toi[0])

Personal Auto Combinations
Recreational Vehicle (RV)
Other
Private Passenger Auto (PPA)
Motorcycle


In [18]:
# input table name in FROM statement to get list of fields
table_name = 'SNL_PC_INSURANCEPRODUCTFILINGS_RATEINFORMATION'
full_table_name = 'XPRESSFEED.' + table_name
fields_query = f"SELECT * FROM {full_table_name} LIMIT 1;"
fields_df = run_sql_query(conn,fields_query,params=None,return_format=1)
for col in fields_df.columns:
    print(col)

OPERATION_0
FILINGCOMPANY_1
SERFFFILINGENTITYEVENTKEY_2
DISPOSITIONDATE_3
RATEFILINGMETHOD_4
RATECHANGETYPE_5
OVERALLPCOFLASTRATEREVISION_6
EFFECTIVEDATEOFLASTRATEREVISION_7
FILINGMETHODOFLASTFILING_8
OVERALLPCINDICATEDCHANGERATEINFO_9
OVERALLPCRATEIMPACTRATEINFO_10
WRITTENPREMIUMCHANGERATEINFO_11
NUMBEROFPOLICYHOLDERSAFFECTEDRATEINFO_12
RATEFILINGWRITTENPREMIUMRATEINFO_13
MAXIMUMPCCHANGERATEINFO_14
MINIMUMPCCHANGERATEINFO_15
MTPLECOOVERALLPERCTRATEINDICATEDRATEINFO_16
MTPLECOOVERALLPERCTRATEIMPACTRATEINFO_17
MTPLECOWRITTENPREMIUMCHANGERATEINFO_18
MTPLECONUMBEROFPOLICYHOLDERSAFFECTEDRATEINFO_19
MTPLECOEARNEDPREMIUMFLONLYRATEINFO_20


# 4 Get data  
- We defined our sql query in a separate file known as rates_query, which we imported

## 4.1 Define sub-type of insurance, start date and end date  
- Let's look at Personal Auto, but exclude Other, Recreational Vehicle (RV) and Motorcycle filings


In [ ]:
sub_type = 'Personal Auto Combinations','Private Passenger Auto (PPA)'

In [13]:
# limit to rate filings only by specifying filing type contains 'Rat'
rates_df = run_sql_query(conn,rates_query,params=('2024-01-01','2024-12-31',sub_type,"%Rat%"),return_format=1)

In [ ]:
# close the database connection
conn.close()